<a href="https://colab.research.google.com/github/rashidmeras/EVA5/blob/master/EVA5_Submission_Session_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

EVA5 Phase1 Assignment 4:

Objective: To train the network and achieve the targeted accuracy with the constranints mentioned below:

* 99.4% validation accuracy
* Less than 20k Parameters
* You can use anything from above you want. 
* Less than 20 Epochs
* No fully connected layer




Import `pytorch` libraries and modules 

In [ ]:
from __future__ import print_function
import torch
#import nueral net model in the torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
#torchvision is the libraray in torch that has many features
#and we are using datasets and transforms from this library 
from torchvision import datasets, transforms

Define the structure of the network

In [8]:
# create the instance of the class
class Net(nn.Module):

    #construct the class
    def __init__(self):
       
        #call parent class and initilize
        super(Net, self).__init__()

        #Layer1: Cov->BN->DO i/p:|28x28x1|Conv(3x3x1)x8| o/p:|26x26x8|   
        self.conv1 = nn.Sequential (
            nn.Conv2d(1, 32, 3),
            nn.ReLU()
            ,nn.BatchNorm2d(32)
            ,nn.Dropout(0.1)
        )

        #Layer2: Cov->BN->DO i/p:|26x26x8|Conv(3x3x8)x4| o/p:|24x24x4|    
        self.conv2 = nn.Sequential (
            nn.Conv2d(32, 16, 3),
            nn.ReLU()
            ,nn.BatchNorm2d(16)
            ,nn.Dropout(0.1)
        )
 
        #Layer3: Cov->BN->DO i/p:|24x24x4|Conv(3x3x4)x4| o/p:|22x22x4|    
        self.conv3 = nn.Sequential (
            nn.Conv2d(16, 16, 3),
            nn.ReLU()
            ,nn.BatchNorm2d(16)
            ,nn.Dropout(0.1)
        )

        #Layer4: Cov->BN->DO i/p:|22x22x4|Conv(1x1x4)x3| o/p:|22x22x3|    
        self.conv4 = nn.Sequential (
            nn.Conv2d(16, 12, 1),
            nn.ReLU()
            ,nn.BatchNorm2d(12)
            ,nn.Dropout(0.1)
        )        

        # Layer5 - Max-Pooling layer 
        self.pool1 = nn.MaxPool2d(2, 2)

        #Layer6: Cov->BN->DO i/p:|11x11x3|Conv(3x3x3)x16| o/p:|9x9x16|
        self.conv6 = nn.Sequential (
            nn.Conv2d(12, 16, 3),
            nn.ReLU()
            ,nn.BatchNorm2d(16)
            ,nn.Dropout(0.1)
        )

        #Layer7: Cov->BN->DO i/p:|9x9x16|Conv(3x3x16)x16| o/p:|7x7x16|
        self.conv7 = nn.Sequential (
            nn.Conv2d(16, 16, 3),
            nn.ReLU()
            ,nn.BatchNorm2d(16)
            ,nn.Dropout(0.1)
        )

        #Layer8: Cov->BN->DO i/p:|7x7x16|Conv(7x7x16)x10| o/p:|1x1x10|
        self.conv8 = nn.Sequential (
            nn.Conv2d(16, 10, 7)
            ,nn.BatchNorm2d(10)
            ,nn.Dropout(0.1)
        )        

    # defines the strcuture of the class
    def forward(self, x):
       
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.conv3(x)
        x = self.conv4(x)
        x = self.pool1(x)
        x = self.conv6(x)
        x = self.conv7(x)
        x = self.conv8(x)

        x = x.view(-1, 10)
        return F.log_softmax(x, dim=1)

Setup to print model summary

In [9]:
!pip install torchsummary
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")

model = Net().to(device)
summary(model, input_size=(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 32, 26, 26]             320
              ReLU-2           [-1, 32, 26, 26]               0
       BatchNorm2d-3           [-1, 32, 26, 26]              64
           Dropout-4           [-1, 32, 26, 26]               0
            Conv2d-5           [-1, 16, 24, 24]           4,624
              ReLU-6           [-1, 16, 24, 24]               0
       BatchNorm2d-7           [-1, 16, 24, 24]              32
           Dropout-8           [-1, 16, 24, 24]               0
            Conv2d-9           [-1, 16, 22, 22]           2,320
             ReLU-10           [-1, 16, 22, 22]               0
      BatchNorm2d-11           [-1, 16, 22, 22]              32
          Dropout-12           [-1, 16, 22, 22]               0
           Conv2d-13           [-1, 12, 22, 22]             204
             ReLU-14           [-1, 12,

Load the testing and training data

In [10]:
torch.manual_seed(1)
batch_size = 128

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)


In [11]:
from tqdm import tqdm
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    pbar = tqdm(train_loader)
    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        pbar.set_description(desc= f'loss={loss.item()} batch_id={batch_idx}')


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.3f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

Start tarining

In [15]:
# model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

for epoch in range(1, 10):
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

  0%|          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 0.0197, Accuracy: 9941/10000 (99.410%)



  0%|          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 0.0193, Accuracy: 9943/10000 (99.430%)



  0%|          | 0/469 [00:00<?, ?it/s]

...


Test set: Average loss: 0.0186, Accuracy: 9946/10000 (99.460%)



  0%|          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 0.0193, Accuracy: 9948/10000 (99.480%)



loss=0.09504666179418564 batch_id=468: 100%|██████████| 469/469 [00:23<00:00, 19.78it/s]



Test set: Average loss: 0.0200, Accuracy: 9945/10000 (99.450%)



## Summary:

A network was modelled and trained and an accuracy of 99.44% was achieved in 20 epochs. The network was designed with the constranints mentioned below:

* 99.4% validation accuracy
* Less than 20k Parameters
* You can use anything from above you want. 
* Less than 20 Epochs
* No fully connected layer


# Results:
* Total params: 19,618
* Epochs = 10
* Accuracy = `99.45%`

